# INCLINOMETROS

In [12]:
import pandas as pd
import os
import numpy as np
import random
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from pylatex import Document, NoEscape,Package
import uuid
from os import remove
from datetime import datetime


In [13]:
DATE_FORMAT = "%d-%m-%y"

In [14]:
# # import locale
# # locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')

# class PlotConfig:
#     """Class to configure global matplotlib parameters."""

#     @classmethod
#     def setup_matplotlib(cls, ts_serie=True, ymargin=0.20):

#         """Configure global matplotlib parameters."""
#         # Set system locale
#         plt.rcParams["axes.formatter.use_locale"] = True

#         # Disable interactive mode and set backend to Agg for non-interactive plotting
#         plt.rcParams["backend"] = "Agg"

#         # Set font family
#         plt.rcParams["font.family"] = "Arial"
#         plt.rcParams["font.size"] = 8

#         # Configure figure appearance
#         plt.rcParams["figure.constrained_layout.use"] = True
#         plt.rcParams["figure.constrained_layout.h_pad"] = 0
#         plt.rcParams["figure.constrained_layout.hspace"] = 0
#         plt.rcParams["figure.constrained_layout.w_pad"] = 0
#         plt.rcParams["figure.constrained_layout.wspace"] = 0
#         plt.rcParams["figure.edgecolor"] = "None"
#         plt.rcParams["figure.facecolor"] = "None"
#         plt.rcParams["figure.titlesize"] = 10
#         plt.rcParams["figure.titleweight"] = "bold"
#         plt.rcParams["figure.autolayout"] = True

#         # Configure axes appearance
#         plt.rcParams["axes.facecolor"] = "None"
#         plt.rcParams["axes.edgecolor"] = "black"
#         plt.rcParams["axes.grid"] = True
#         plt.rcParams["axes.ymargin"] = ymargin
#         plt.rcParams["axes.spines.bottom"] = True
#         plt.rcParams["axes.spines.left"] = True
#         plt.rcParams["axes.spines.right"] = True
#         plt.rcParams["axes.spines.top"] = True
#         plt.rcParams["axes.titleweight"] = "bold"
#         plt.rcParams["axes.titlesize"] = 10
#         plt.rcParams["axes.labelsize"] = 9
#         plt.rcParams["axes.titlepad"] = 4
#         plt.rcParams["axes.titlelocation"] = "center"

#         # Configure date formatting
#         plt.rcParams["date.autoformatter.day"] = DATE_FORMAT
#         plt.rcParams["date.autoformatter.month"] = DATE_FORMAT
#         plt.rcParams["date.autoformatter.year"] = DATE_FORMAT

#         # Configure ticks
#         plt.rcParams["ytick.minor.visible"] = True
#         plt.rcParams["ytick.labelsize"] = 8
#         plt.rcParams["xtick.labelsize"] = 8

#         # Configure legend
#         plt.rcParams["legend.loc"] = "upper left"
#         plt.rcParams["legend.fontsize"] = 8
#         plt.rcParams["legend.facecolor"] = "white"
#         plt.rcParams["legend.framealpha"] = 0.15
#         plt.rcParams["legend.edgecolor"] = "None"
#         plt.rcParams["legend.fancybox"] = False

#         # Configure grid
#         plt.rcParams["grid.alpha"] = 0.25
#         plt.rcParams["grid.color"] = "gray"
#         plt.rcParams["grid.linestyle"] = "-"
#         plt.rcParams["grid.linewidth"] = 0.05

#         if ts_serie:
#             plt.rcParams["axes.xmargin"] = 0
#             plt.rcParams["xtick.minor.visible"] = False
#         else:
#             plt.rcParams["axes.xmargin"] = 0.05
#             plt.rcParams["xtick.minor.visible"] = True


In [15]:
class Inclinometer:
    data = []

    def __init__(self,ruta_carpeta):
        self.archivos_de_texto = sorted([archivo for archivo in os.listdir(ruta_carpeta) if archivo.endswith('.gkn')])
        print(self.archivos_de_texto)
        df_data = pd.read_csv(ruta_carpeta+"/Datos.csv",sep=";",encoding='utf-8')
        self.client,self.project,self.cod_project,self.component,self.maker,self.reviewer,self.revision,self.graph,self.instrumento,self.sonda,self.marca,self.estado,self.este,self.norte,self.cota,self.Acimut_incl,self.Acimut_talud,self.nivel_moderado,self.nivel_critico,self.nivel_max=df_data["TEXTO"]
        self.nivel_moderado = float(self.nivel_moderado)
        self.nivel_critico = float(self.nivel_critico)
        self.nivel_max = float(self.nivel_max)
        self.cota = float(self.cota)
        self.este, self.norte, self.sonda= float(self.este), float(self.norte), float(self.sonda)
        self.constants()
        list_point = [".","v","^","s","*","x","d","p","P","H","+"]
        for nombre_archivo in self.archivos_de_texto:
            ruta_archivo = os.path.join(ruta_carpeta, nombre_archivo)
            # Abre y lee el archivo de texto
            with open(ruta_archivo, 'r') as archivo:
                contenido = archivo.readlines()
            date = contenido[4].split(":")[-1][:-1]
            date = date.split("/")
            date = [str(date[1]),str(date[0]),str(date[2])]
            date = "/".join(date)
            if nombre_archivo==self.archivos_de_texto[0]:
                self.medida_base(ruta_archivo)
                # Corrige: convierte date (string) a formato dd-mm-yy si es posible
                try:
                    dt = datetime.strptime(date, "%m/%d/%y")
                    self.base = dt.strftime("%d-%m-%y")
                except Exception:
                    self.base = date
            self.procesamiento(ruta_archivo,date,random.choice(list_point))
        
        self.gra_def = {"A+":{"fig":""},"B+":{"fig":""}}

    def constants(self):
        self.ΔL = 0.5
        if self.marca == "GEOKON":    
            self.BK1 = 0.005
        else:
            self.BK1 = 0.02
        self.BJ2 = 0.005
        self.BG2 = (float(self.Acimut_incl)-float(self.Acimut_talud))*np.pi/180

    def medida_base(self,file):
        df = pd.read_csv(file, skiprows=10, names=["FLEVEL", "A+", "A-", "B+", "B-"])
        df["A+"] = np.array(df["A+"])[::-1]
        df["A-"] = np.array(df["A-"])[::-1]
        df["B+"] = np.array(df["B+"])[::-1]
        df["B-"] = np.array(df["B-"])[::-1]
        df["L"] = (df["FLEVEL"]!=0)*self.ΔL
        df["DIF(A)/2"] = (df["A+"]-df["A-"])/2
        df["DIF(B)/2"] = (df["B+"]-df["B-"])/2
        df["A check"] = (df["A+"]+df["A-"])/2
        df["B check"] = (df["B+"]+df["B-"])/2
        df["A (cm)"] = df["L"]*df["DIF(A)/2"]*self.BK1
        df["B (cm)"] = df["L"]*df["DIF(B)/2"]*self.BJ2
        df["A Deflex. (cm) Sin corregir"] = df["A (cm)"][::-1].cumsum()[::-1]
        df["B Deflex. (cm) Sin corregir"] = df["B (cm)"][::-1].cumsum()[::-1]
        df["A despl. (cm)"] = df["A Deflex. (cm) Sin corregir"]*np.cos(self.BG2)-df["B Deflex. (cm) Sin corregir"]*np.sin(self.BG2)
        df["B despl. (cm)"] = df["A Deflex. (cm) Sin corregir"]*np.sin(self.BG2)+df["B Deflex. (cm) Sin corregir"]*np.cos(self.BG2)
        df["Comentarios"] = ["Condición normal" for i in range(len(df))]
        self.df_base = df
    
    def procesamiento(self,file,date,marker):
        df = pd.read_csv(file, skiprows=10, names=["FLEVEL", "A+", "A-", "B+", "B-"])
        df["A+"] = np.array(df["A+"])[::-1]
        df["A-"] = np.array(df["A-"])[::-1]
        df["B+"] = np.array(df["B+"])[::-1]
        df["B-"] = np.array(df["B-"])[::-1]
        df["L"] = (df["FLEVEL"]!=0)*self.ΔL
        df["DIF(A)/2"] = (df["A+"]-df["A-"])/2
        df["DIF(B)/2"] = (df["B+"]-df["B-"])/2
        df["A check"] = (df["A+"]+df["A-"])/2
        df["B check"] = (df["B+"]+df["B-"])/2
        df["A (cm)"] = df["L"]*df["DIF(A)/2"]*self.BK1
        df["B (cm)"] = df["L"]*df["DIF(B)/2"]*self.BJ2
        df["A Deflex. (cm) Sin corregir"] = df["A (cm)"][::-1].cumsum()[::-1]
        df["B Deflex. (cm) Sin corregir"] = df["B (cm)"][::-1].cumsum()[::-1]
        df["A despl. (cm)"] = df["A Deflex. (cm) Sin corregir"]*np.cos(self.BG2)-df["B Deflex. (cm) Sin corregir"]*np.sin(self.BG2)
        df["B despl. (cm)"] = df["A Deflex. (cm) Sin corregir"]*np.sin(self.BG2)+df["B Deflex. (cm) Sin corregir"]*np.cos(self.BG2)
        df["A deflex. (cm)"] = df["A despl. (cm)"]-self.df_base["A despl. (cm)"]
        df["B deflex. (cm)"] = df["B despl. (cm)"]-self.df_base["B despl. (cm)"]

        list_comentarios = []
        for i in range(len(df)):
            if abs(df["A deflex. (cm)"])[i] > abs(df["B deflex. (cm)"])[i]:
                if abs(df["A deflex. (cm)"])[i]>self.nivel_critico:
                    list_comentarios.append("Nivel de alerta crítico en A")
                elif abs(df["A deflex. (cm)"])[i]>self.nivel_moderado:
                    list_comentarios.append("Nivel de alerta moderado en A")
                else:
                    list_comentarios.append("Condición normal")
            else:
                if abs(df["B deflex. (cm)"])[i]>self.nivel_critico:
                    list_comentarios.append("Nivel de alerta crítico en B")
                elif abs(df["B deflex. (cm)"])[i]>self.nivel_moderado:
                    list_comentarios.append("Nivel de alerta moderado en B")
                else:
                    list_comentarios.append("Condición normal")

        df["Comentarios"] = list_comentarios

        # Convertir la fecha a formato dd-mm-yy para el nombre de la serie
        try:
            # date viene como 'mm/dd/yy' o similar, convertir a datetime y luego a string
            dt = None
            for fmt in ("%m/%d/%y", "%d/%m/%y", "%d-%m-%y", "%Y-%m-%d"):
                try:
                    dt = datetime.strptime(date, fmt)
                    break
                except Exception:
                    continue
            if dt is None:
                # Si no se pudo convertir, dejar el string original
                date_fmt = date
            else:
                date_fmt = dt.strftime("%d-%m-%y")
        except Exception:
            date_fmt = date

        incl = {"date": date_fmt, "df": df, "mark": marker}
        self.data.append(incl)

    def latex_escape(self, text):
        """
        Escapa caracteres especiales de LaTeX en un string.
        Permite tildes y eñes, y reduce el tamaño de letra si el texto es largo.
        """
        if not isinstance(text, str):
            text = str(text)
        # Permitir tildes y eñes, no tocar letras unicode
        # Solo escapar caracteres LaTeX problemáticos
        text = text.replace('\\', r'\textbackslash{}')
        replacements = {
            '&': r'\&',
            '%': r'\%',
            '$': r'\$',
            '#': r'\#',
            '_': r'\_',
            '{': r'\{',
            '}': r'\}',
            '~': r'\textasciitilde{}',
            '^': r'\^{}',
        }
        for char, replacement in replacements.items():
            text = text.replace(char, replacement)
        # Si el texto es largo, reducir el tamaño de letra
        if len(text) > 50:
            # Puedes ajustar el umbral y el tamaño según necesidad
            text = r'\scriptsize{' + text + '}'
        return text

    def data_excel(self):
        import openpyxl
        from openpyxl.drawing.image import Image
        from openpyxl.drawing.spreadsheet_drawing import AbsoluteAnchor
        from openpyxl.drawing.xdr import XDRPoint2D, XDRPositiveSize2D
        from openpyxl.utils.units import pixels_to_EMU, cm_to_EMU
        from openpyxl.utils.dataframe import dataframe_to_rows
        from openpyxl.styles import Font, Alignment, Border, Side, PatternFill

        p2e = pixels_to_EMU
        archivo_excel = openpyxl.load_workbook('Formato_INC.xlsx')
        primera_hoja = archivo_excel.active
        area_de_impresion = primera_hoja.print_area

        for inc in self.data:
            nueva_hoja = archivo_excel.copy_worksheet(primera_hoja)
            nueva_hoja.title = inc["date"].replace("/","-")
            print(nueva_hoja.title)
            nueva_hoja["C6"].value = self.project
            nueva_hoja["C7"].value = self.cod_project
            nueva_hoja["C8"].value = self.client
            nueva_hoja["M6"].value = self.component
            nueva_hoja["M7"].value = self.maker
            nueva_hoja["M8"].value = self.reviewer
            nueva_hoja["N2"].value = self.revision
            nueva_hoja["N3"].value = "=TODAY()"
            nueva_hoja["N10"].value = self.este
            nueva_hoja["N11"].value = self.norte
            nueva_hoja["D10"].value = self.instrumento
            nueva_hoja["D11"].value = self.sonda
            nueva_hoja["D12"].value = self.marca
            
            nueva_hoja["D14"].value = datetime.strptime(str(self.base),"%d/%m/%y")#.strftime("%d/%m/%Y")
            # nueva_hoja["D15"].value = datetime.strptime(str(inc["date"]),"%d/%m/%y")#.strftime("%d/%m/%Y")
            nueva_hoja["I10"].value = self.cota
            nueva_hoja["I11"].value = self.cota - inc["df"]["FLEVEL"][0]
            nueva_hoja["I12"].value = inc["df"]["FLEVEL"][0]

            imagen = Image('Logo.png')
            #Specify image hight and width
            imagen.width = 245.57
            imagen.height  = 53.59
            #Define x and y position in the sheet
            position = XDRPoint2D(p2e(11), p2e(10))
            size = XDRPositiveSize2D(p2e(imagen.width), p2e(imagen.height))
            imagen.anchor = AbsoluteAnchor(pos=position, ext=size)
            nueva_hoja.add_image(imagen)

            df = inc["df"][["FLEVEL","L","A+", "A-", "B+", "B-","A check","B check","A despl. (cm)","B despl. (cm)","A deflex. (cm)","B deflex. (cm)","Comentarios"]].copy()
            print(df)
            start_row = 18
            start_col = 1
            for fila_datos in dataframe_to_rows(df, index=False, header=False):
                for dato in fila_datos:
                    celda = nueva_hoja.cell(row=start_row, column=start_col, value=dato)
                    celda.font = Font(name='Arial', bold=False, size=10)
                    celda.border = Border(left=Side(style="thin"), right=Side(style="thin"), top=Side(style="thin"), bottom=Side(style="thin"))
                    celda.alignment = Alignment(horizontal="center", vertical="center")

                    if start_col == 1:
                        celda.number_format = '0.00'
                        celda.fill = PatternFill(start_color="BFBFBF", end_color="BFBFBF", fill_type="solid")
                    elif start_col == 2:
                        celda.number_format = '0.00'
                    elif 2<start_col<=6:
                        celda.number_format = '0'
                        celda.fill = PatternFill(start_color="BFBFBF", end_color="BFBFBF", fill_type="solid")
                    elif start_col == 13:
                        nueva_hoja.merge_cells("M"+str(start_row)+":O"+str(start_row))
                        celda_combinada = nueva_hoja["M"+str(start_row)]
                        celda_combinada.border = Border(left=Side(style="thin"), right=Side(style="thin"), top=Side(style="thin"), bottom=Side(style="thin"))
                    else:
                        celda.number_format = '0.00'

                    start_col += 1
                start_row += 1
                start_col = 1
            
            nueva_hoja.print_area = "A1:O"+str(start_row-1)
            nueva_hoja.freeze_panes = nueva_hoja["P18"]
            nueva_hoja.print_title_rows = '1:17'
            nueva_hoja.sheet_view.view = 'pageBreakPreview'

        hoja_base = archivo_excel.worksheets[1]
        hoja_base.title = "MedidadBase"
        for i in range(18,start_row):
            hoja_base.cell(row=i, column=11, value="-")
            hoja_base.cell(row=i, column=12, value="-")

        archivo_excel.remove(primera_hoja)
        archivo_excel.save('data_inc_tables.xlsx')
        archivo_excel.close()
    
    def plot_deflexion(self,type):
        self.gra_def[type]["fig"] = Figure(figsize=(18,14),dpi=100)
        ax = self.gra_def[type]["fig"].add_subplot(111)

        # fig,ax = plt.subplots(figsize=(18,14),dpi=100)

        if type == "A+":
            text = "A deflex. (cm)"
        else:
            text = "B deflex. (cm)"

        for incl in self.data:
            ax.plot(incl["df"][text],incl["df"]["FLEVEL"],label=incl["date"],alpha=1,marker=incl["mark"])

        # ax.axvline(self.nivel_moderado,color="green",linestyle="-",label="Nivel de alerta normal")
        # ax.axvline(self.nivel_critico,color="yellow",linestyle="-",label="Nivel de alerta moderado")
        # ax.axvline(self.nivel_max,color="red",linestyle="-",label="Nivel de alerta crítico")
        # ax.axvline(-self.nivel_moderado,color="green",linestyle="-")
        # ax.axvline(-self.nivel_critico,color="yellow",linestyle="-")
        # ax.axvline(-self.nivel_max,color="red",linestyle="-")

        # ax.axvspan(0,self.nivel_moderado,color="green",alpha=0.2)
        # ax.axvspan(-self.nivel_moderado,0,color="green",alpha=0.2)
        # ax.axvspan(self.nivel_moderado,self.nivel_critico,color="yellow",alpha=0.2)
        # ax.axvspan(-self.nivel_critico,-self.nivel_moderado,color="yellow",alpha=0.2)
        # ax.axvspan(self.nivel_critico,self.nivel_max,color="red",alpha=0.2)
        # ax.axvspan(-self.nivel_max,-self.nivel_critico,color="red",alpha=0.2)
        # ax.axvline(0,color="black",linestyle="--")

        ax.set_xlabel("Deflexión (cm)",fontsize=14,fontname="Arial",labelpad=20)
        ax.set_ylabel("Longitud de tubería inclinométrica (m)",fontsize=14,fontname="Arial",labelpad=10)
        ax.legend(fontsize=10,loc="upper left",ncol=2,edgecolor="black",facecolor="white",bbox_to_anchor=(1.05, 1.0),frameon=False)
        ax.set_title("Registro histórico de inclinómetro vertical",fontsize=18,pad=20,fontname="Arial")
        ax.set_xticks(np.linspace(-self.nivel_max,self.nivel_max,21))
        ax.set_xlim(-self.nivel_max,self.nivel_max)
        ax.set_ylim(0,incl["df"]["FLEVEL"][0])
        ax.grid()
    
    def generate_pdf(self,fec,cod):

        strr = r"""
\date{}
\pagestyle{empty}
\thispagestyle{empty}
"""+"""\n"""
        ni,nd = self.graph.split(".")
        nd = int(nd)

        # Escapar campos de texto para LaTeX
        cod_latex = self.latex_escape(cod)
        client_latex = self.latex_escape(self.client)
        maker_latex = self.latex_escape(self.maker)
        # Para 'Proyecto', permitir tildes y reducir tamaño si es largo
        project_latex = self.latex_escape(self.project)
        reviewer_latex = self.latex_escape(self.reviewer)
        revision_latex = self.latex_escape(self.revision)
        component_latex = self.latex_escape(self.component)
        instrumento_latex = self.latex_escape(self.instrumento)
        base_latex = self.latex_escape(self.base)

        for type in ["A+","B+"]:
            self.gra_def[type]["path"] = str(uuid.uuid4())+".pdf"
            self.gra_def[type]["fig"].savefig(self.gra_def[type]["path"],pad_inches=0,bbox_inches='tight',format="pdf")
            
            sub = "Deflexiones acumuladas en dirección "+type+" del inclinómetro "+instrumento_latex+" con medida base ("+base_latex+") - "+component_latex

            strr_ = r"""
\begin{table}[htbp]
  \centering
    \begin{tabular}{|>{\centering\arraybackslash}m{0.2\textwidth}|m{0.1\textwidth}|m{0.1\textwidth}|>{\centering\arraybackslash}m{0.52\textwidth}|>{\centering\arraybackslash}m{0.08\textwidth}|}
    \hline
    \multicolumn{5}{|c|}{\rule{0pt}{15.6cm} \multirow{1}{*}[15cm]{ \includegraphics[height=15cm]{"""+self.gra_def[type]["path"]+r"""}}}\\
    \hline
    \multirow{4}{*}{\includegraphics[height=1.4cm]{anddes.pdf}} & \textbf{\footnotesize N° Proyecto} & \textbf{\footnotesize """+cod_latex+r"""} & \textbf{\footnotesize """+client_latex+r"""}  & {\small GRÁFICA N°} \\
\cline{2-5}          & \textbf{\footnotesize Elaborado} & \textbf{\footnotesize """+maker_latex+r"""}  & \textbf{\footnotesize """+project_latex+r"""} & \multirow{4}{0.08\textwidth}{\centering \Huge """+ni+"."+str(nd)+r"""} \\
\cline{2-4}          & \textbf{\footnotesize Aprobado} &  \textbf{\footnotesize """+reviewer_latex+r"""}   & \multirow{3}{12cm}{\centering \small """+sub+r"""}      &  \\
\cline{2-3}          & \textbf{\footnotesize Fecha} &   \textbf{\footnotesize """+fec+r"""}    &  &  \\
\cline{1-3}   \cellcolor[rgb]{ 0,  .439,  .753} \textcolor[rgb]{ 1,  1,  1}{\footnotesize SIG AND} & \textbf{\footnotesize Revisión} & \textbf{\footnotesize """+revision_latex+r"""} &       &  \\
    \hline
    \end{tabular}%
\end{table}%
"""+"\n"
            strr = strr + strr_
            nd = nd + 1
        
        geometry_options = {"right": "3.8cm", "left": "1.5cm","top":"1.5cm","bottom":"1.5cm","landscape":True,"a4paper":True}
        self.report_name = str(uuid.uuid4()).split("-")[0]
        self.report_name_path= self.report_name
        doc = Document(self.report_name,geometry_options=geometry_options)
        doc.packages.append(Package('multirow'))
        doc.packages.append(Package('array'))
        doc.packages.append(Package('colortbl'))
        doc.packages.append(Package('booktabs'))
        doc.packages.append(Package('graphicx'))
        doc.packages.append(Package('helvet'))
        doc.preamble.append(NoEscape(r'\renewcommand{\familydefault}{\sfdefault}'))

        doc.append(NoEscape(strr))
        doc.generate_pdf(compiler="pdflatex")

        for type in ["A+","B+"]:
            remove(self.gra_def[type]["path"])

In [16]:
# Especifica la ruta de la carpeta que deseas explorar
ruta_carpeta = "/home/joelefrain/Desktop/_workspace/geo_sentry/seed/sample_client/sample_project/250228_Febrero/DME Choloque/INCLINOMETRO/INC-101"
data_incl = Inclinometer(ruta_carpeta)

data_incl.plot_deflexion("A+")
data_incl.plot_deflexion("B+")

# data_incl.data_excel()
data_incl.generate_pdf("15-05-25","1410.28.0054-0000")


['101-054A.gkn', '101-10A.gkn', '101-11A.gkn', '101-12A.gkn', '101-13A.gkn', '101-14A.gkn', '101-15A.gkn', '101-16A.gkn', '101-17A.gkn', '101-18A.gkn', '101-19A.gkn', '101-1A.gkn', '101-20A.gkn', '101-21A.gkn', '101-22A.gkn', '101-23A.gkn', '101-24A.gkn', '101-25A.gkn', '101-26A.gkn', '101-27A.gkn', '101-28A.gkn', '101-29A.gkn', '101-2A.gkn', '101-30A.gkn', '101-31A.gkn', '101-32A.gkn', '101-33A.gkn', '101-36A.gkn', '101-37A.gkn', '101-38A.gkn', '101-39A.gkn', '101-3A.gkn', '101-40A.gkn', '101-41A.gkn', '101-42A.gkn', '101-43A.gkn', '101-44A.gkn', '101-46A.gkn', '101-47A.gkn', '101-48A.gkn', '101-49A.gkn', '101-4A.gkn', '101-50A.gkn', '101-51A.gkn', '101-52A.gkn', '101-53A.gkn', '101-55A.gkn', '101-56A.gkn', '101-57A.gkn', '101-58A.gkn', '101-59A.gkn', '101-5A.gkn', '101-60A.gkn', '101-61A.gkn', '101-62A.gkn', '101-63A.gkn', '101-64A.gkn', '101-65A.gkn', '101-66A.gkn', '101-67A.gkn', '101-68A.gkn', '101-69A.gkn', '101-6A.gkn', '101-70A.gkn', '101-71A.gkn', '101-73A.gkn', '101-74A.gkn',

TypeError: unsupported operand type(s) for -: 'str' and 'str'